### Text Generation Script

In [8]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import sys

# Load the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('./myLM')
model = GPT2LMHeadModel.from_pretrained('./myLM_gpt2')

# Define the pad_token as eos_token
tokenizer.pad_token = tokenizer.eos_token

# Update the model's configuration to recognize the pad_token_id
model.config.pad_token_id = tokenizer.pad_token_id

# Configure device
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS backend")
else:
    device = torch.device("cpu")
    print("Using CPU")

model.to(device)
model.eval()

def generate_text(prompt, max_length=100):
    # Encode the prompt and generate attention_mask
    encoded = tokenizer(
        prompt,
        return_tensors='pt',
        padding=False,  # No padding needed for single inputs
        truncation=True,
        max_length=512  # Adjust based on model's max input length
    )
    input_ids = encoded['input_ids'].to(device)
    attention_mask = encoded['attention_mask'].to(device)

    with torch.no_grad():
        output = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,  # Pass the attention_mask
            max_length=max_length,
            num_beams=1,
            no_repeat_ngram_size=0,
            early_stopping=True,
            temperature=1.9,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id  # Ensure pad_token_id is set
        )
    return tokenizer.decode(output[0], skip_special_tokens=True)

def main():
    while True:
        prompt = input("Enter a prompt (or type 'exit' to quit): ")
        if prompt.lower() == 'exit':
            break
        print(f"Prompt: {prompt}")
        print(generate_text(prompt)[len(prompt):], sep="")
        print("\n" + "-"*60 + "\n")

if __name__ == "__main__":
    if "ipykernel_launcher" in sys.argv[0]:
        main()
    else:
        # Running as a standalone script
        parser = argparse.ArgumentParser(description="Generate text based on a prompt.")
        parser.add_argument("--prompt", nargs="?", default=None, help="The prompt to generate text from.")
        args = parser.parse_args()
        main()

Using MPS backend


Enter a prompt (or type 'exit' to quit):  this is my home


Prompt: this is my home
 in the quiet and wonderful North Adams--two stories from I'll give you an easy commute if you need it, 2 stories from the Tacoma media area.
I work from home two days a week, one out of respect for myself, and two to keep the house clean and orderly. I keep all common areas clean as a group. I expect my roommates to be respectful and conscientious, pretty quiet at times.
Not a partying cat, but I do occasionally have a

------------------------------------------------------------



Enter a prompt (or type 'exit' to quit):  exit
